In [7]:
import pandas as pd
import glob

In [8]:
file_paths = glob.glob('data/bonus/*.xlsx')


In [9]:
df = pd.DataFrame()

In [10]:
for file in file_paths:
    filename = file.split("\\")[1].split('.')[0]
    numbeer = filename[:10]
    name = filename[10:]
    temp_df = pd.read_excel(file)
    

In [11]:
test = pd.read_excel('data\\bonus\\2024123456倪小丰.xlsx')
test.head()

,学号,姓名,项目,加分,项目.1,加分.1,项目.2,加分.2,德育汇总（5分）,项目.3,加分.3,智育汇总（5分）,体育汇总（3分）,美育汇总（2分）,劳育汇总（2分）,所有拔尖项汇总,备注
0,2024123456,倪小丰,校级_2025_优秀团员,0.5,2025_行政班班长,0.5,2025_知行杯_参与结项,0.5,1.5,国家级_2025_大学生计算机设计_三等奖_团队（3人）_组员,1.6,1.6,0,0,0,3.1,NaN


In [12]:
test.columns

Index(['学号', '姓名', '项目', '加分', '项目.1', '加分.1', '项目.2', '加分.2', '德育汇总（5分）',
       '项目.3', '加分.3', '智育汇总（5分）', '体育汇总（3分）', '美育汇总（2分）', '劳育汇总（2分）',
       '所有拔尖项汇总', '备注'],
      dtype='object')

In [13]:
total_dict = {}

sections = [
    ("德育", "德育汇总（5分）"),
    ("智育", "智育汇总（5分）"),
    ("体育", "体育汇总（3分）"),
    ("美育", "美育汇总（2分）"),
    ("劳育", "劳育汇总（2分）")
]

total_dict = {}

for file in file_paths:
    # print(file)
    temp_df = pd.read_excel(file)
    row = temp_df.iloc[0]
    student_id = row['学号']
    student_name = row['姓名']

    # 获取各板块汇总分
    section_scores = {}
    for section, col in sections:
        section_scores[section] = row.get(col, None)

    # 按板块分类加分项
    section_bonus = {sec: [] for sec, _ in sections}
    col_list = temp_df.columns.tolist()
    # 找到各板块汇总列的索引
    section_indices = [col_list.index(col) if col in col_list else None for _, col in sections]

    for i in range(len(col_list)):
        col = col_list[i]
        if col.startswith("项目"):
            bonus_col = col.replace("项目", "加分")
            item = row[col]
            bonus = row.get(bonus_col, None)
            if pd.notna(item) and pd.notna(bonus):
                # 判断属于哪个板块
                for idx, (sec, _) in enumerate(sections):
                    start = section_indices[idx - 1] + 1 if idx > 0 else 0
                    end = section_indices[idx]
                    if start <= i < end:
                        section_bonus[sec].append((item, bonus))
                        break

    total_dict[student_id] = {
        "姓名": student_name,
        "加分项": section_bonus,
        "板块总分": section_scores
    }

In [14]:
sections = [
    ("德育", "德育汇总（5分）"),
    ("智育", "智育汇总（5分）"),
    ("体育", "体育汇总（3分）"),
    ("美育", "美育汇总（2分）"),
    ("劳育", "劳育汇总（2分）")
]

total_dict = {}

for file in file_paths:
    temp_df = pd.read_excel(file)
    row = temp_df.iloc[0]
    student_id = row['学号']
    student_name = row['姓名']

    section_scores = {}
    for section, col in sections:
        val = row.get(col, 0)
        section_scores[section] = 0 if pd.isna(val) else val

    section_bonus = {sec: [] for sec, _ in sections}
    col_list = temp_df.columns.tolist()
    section_indices = [col_list.index(col) if col in col_list else None for _, col in sections]

    for i in range(len(col_list)):
        col = col_list[i]
        if col.startswith("项目"):
            item = row[col]
            if pd.isna(item):
                continue  # 项目缺失直接跳过
            bonus_col = col.replace("项目", "加分")
            bonus = row.get(bonus_col, None)
            if pd.notna(bonus):
                for idx, (sec, _) in enumerate(sections):
                    start = section_indices[idx - 1] + 1 if idx > 0 else 0
                    end = section_indices[idx]
                    if start <= i < end:
                        section_bonus[sec].append((item, bonus))
                        break

    total_dict[student_id] = {
        "姓名": student_name,
        "加分项": section_bonus,
        "板块总分": section_scores,
        "总分": sum(section_scores.values())
    }

In [15]:
# total_dict

In [16]:
import numpy as np

max_bonus_counts = {}
for sec, _ in sections:
    max_bonus_counts[sec] = max(len(total_dict[sid]['加分项'][sec]) for sid in total_dict)

columns = ['学号', '姓名']
for sec, col_name in sections:
    for i in range(1, max_bonus_counts[sec] + 1):
        columns.append(f"{sec}项目{i}")
        columns.append(f"{sec}分数{i}")
    columns.append(col_name)
columns += ['所有拔尖项汇总', '备注']

data = []
for sid, info in total_dict.items():
    row = [sid, info['姓名']]
    for sec, col_name in sections:
        bonus_items = info['加分项'][sec]
        for i in range(max_bonus_counts[sec]):
            if i < len(bonus_items):
                row.append(bonus_items[i][0])
                row.append(bonus_items[i][1])
            else:
                row.append(np.nan)
                row.append(np.nan)
        row.append(info['板块总分'][sec])
    row.append(info['总分'])
    row.append('')
    data.append(row)

summary_df = pd.DataFrame(data, columns=columns)

In [17]:
summary_df.shape

(2, 17)

In [18]:
standard_columns = pd.read_excel('data/Part2.xlsx')

In [19]:
all_numbers = standard_columns['学号'].tolist()
# 转专业同学学号
drop_list = [2024111155, 2024111279, 2024110220, 2024110888, 2024111547, 2024111923, 2024110867]

In [20]:
summary_df = summary_df[~summary_df['学号'].isin(drop_list)]
summary_df.shape

(2, 17)

In [21]:
not_updated = [x for x in all_numbers if x not in summary_df['学号'].tolist()]

In [22]:
standard_columns[standard_columns['学号'].isin(not_updated)]

,学号,姓名,班级,均分,门数,总学分,pe1_score,pe2_score,除去体育分数,学业水平分,体育分数,德育成绩


In [23]:
with pd.ExcelWriter('data/bonus_scores.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='加分项（分模块）', index=False)